<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6455a617ac3dddd54a7c558601f75a8da9e4805532c8e025fb9c942c3ec2d12f
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-24 09:16:04
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.41 C
-------------------
Today PnL: -10.01 K (-0.07%)
Current PnL: -19.02 L (-12.79%)
CY Booked + Current PnL: -7.58 L (-5.1%)
-------------------
Total profit:  1.51 L
Total loss:  -20.53 L
-------------------
Total Booked + Current PnL: 19.40 L (15.94%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.14%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 84.89 L (60.39%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 8.82%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,12.85,57.0,H-MC,3.77,194700.0,30030.0,13298.0,-0.87,18.24,6.83,26.31,89.0,2.26,1.39,27.67,X40N,NTT,AC
77,TTKPRESTIG,770.00,97.23,41.0,M-SC,2.94,84967.0,-15810.0,15906.0,-0.12,-15.69,18.72,0.09,245.0,-0.99,0.60,11.63,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-44.49,60.0,H-MC,9.90,110828.0,10324.0,18785.0,0.13,10.27,16.95,28.96,79.0,0.55,0.79,47.73,MH,ATH,METALS
57,RELIANCE,1533.00,-13.66,47.0,H-LC,4.45,216200.0,5834.0,22939.0,-0.28,2.77,10.61,13.68,37.0,0.25,1.54,19.75,XY25,NTT,REFINERIES
84,VOLTAS,1530.00,-1.92,46.0,H-MC,2.97,205830.0,14088.0,23670.0,-0.04,7.35,11.50,19.69,99.0,0.60,1.46,14.64,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1800.0,250.84,49.0,L-MC,12.24,43049.0,-34158.0,59550.0,2.85,-44.24,138.33,32.89,258.0,-0.57,0.31,18.55,XR,NTT,BANKS
21,COFFEEDAY,80.0,-41.65,54.0,L-SC,42.83,86400.0,-27149.0,67202.0,1.10,-23.91,77.78,35.27,268.0,-0.40,0.61,111.47,XR,NTT,HOTELS
55,RAJOOENG,143.1,-38.17,34.0,H-SC,12.79,93580.0,-8920.0,49523.0,0.66,-8.70,52.92,39.61,114.0,-0.18,0.67,0.66,AR,ATH,MISC
56,RELAXO,1176.0,-36.78,41.0,H-SC,2.27,86142.0,-59018.0,134950.0,0.66,-40.66,156.66,52.31,136.0,-0.44,0.61,15.31,X40N,NTT,FOOTWEAR
11,BANKINDIA,190.0,-26.71,70.0,H-MC,7.47,187836.0,8028.0,104380.0,0.58,4.47,55.57,62.52,88.0,0.08,1.34,40.07,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-14.63,39.0,H-LC,7.65,215327.0,-36441.0,89641.0,-1.44,-14.47,41.63,21.13,27.0,-0.41,1.53,15.82,X40,ATH,PAINTS
82,VALIANTORG,838.00,-313.35,43.0,H-SC,5.02,118660.0,-44945.0,166255.0,-1.19,-27.47,140.11,74.15,139.0,-0.27,0.84,52.10,XR,NTT,CHEMICALS
14,BLUESTARCO,2080.00,12.85,57.0,H-MC,3.77,194700.0,30030.0,13298.0,-0.87,18.24,6.83,26.31,89.0,2.26,1.39,27.67,X40N,NTT,AC
6,ASIANTILES,137.00,6894.44,57.0,L-SC,5.35,76717.0,-17093.0,93710.0,-0.79,-18.22,122.15,81.67,269.0,-0.18,0.55,48.57,XR,NTT,CERAMICS
37,INDIAMART,4850.92,-54.83,26.0,H-SC,10.71,125923.0,2587.0,131174.0,-0.63,2.10,104.17,108.45,119.0,0.02,0.90,24.92,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-21.11,44.0,H-MC,1.99,135736.0,-52.0,42811.0,-0.04,-0.04,31.54,31.48,107.0,-0.00,0.97,12.08,X40,ATH,INSURANCE
23,DABUR,735.00,-7.80,38.0,H-MC,6.81,249751.0,1683.0,106719.0,-0.01,0.68,42.73,43.70,102.0,0.02,1.78,15.22,XY24,BTT,FMCG
52,PGHH,17973.08,-30.41,53.0,H-MC,3.67,201555.0,735.0,68045.0,0.22,0.37,33.76,34.25,80.0,0.01,1.43,5.58,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,50.0,H-SC,9.61,121446.0,-21033.0,40551.0,-0.51,-14.76,33.39,13.70,163.0,-0.52,0.86,43.34,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,97.23,41.0,M-SC,2.94,84967.0,-15810.0,15906.0,-0.12,-15.69,18.72,0.09,245.0,-0.99,0.60,11.63,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,53.0,H-SC,1.13,222327.0,-47340.0,83684.0,0.33,-17.55,37.64,13.47,138.0,-0.57,1.58,12.45,XY24,NTT,PAINTS
18,CERA,9475.0,-22.43,35.0,H-SC,2.13,141450.0,-34453.0,76482.0,-0.49,-19.59,54.07,23.89,149.0,-0.45,1.01,22.16,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-21.86,45.0,H-MC,6.86,105802.0,-26033.0,68253.0,0.23,-19.75,64.51,32.03,98.0,-0.38,0.75,18.51,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1262.04,81.0,M-SC,10.07,185336.0,4193.0,41274.0,0.23,2.31,22.27,25.10,235.0,0.10,1.32,48.53,XY24,NTT,HEALTHCARE
60,SAIL,228.00,49.16,70.0,M-MC,9.75,235862.0,10900.0,156754.0,-0.24,4.85,66.46,74.53,192.0,0.07,1.68,38.66,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,12.85,57.0,H-MC,3.77,194700.0,30030.0,13298.0,-0.87,18.24,6.83,26.31,89.0,2.26,1.39,27.67,X40N,NTT,AC
17,CAMS,5211.76,-1.84,57.0,H-SC,2.18,111636.0,9632.0,34295.0,0.02,9.44,30.72,43.06,122.0,0.28,0.79,28.18,X40N,ATH,MISC
84,VOLTAS,1530.00,-1.92,46.0,H-MC,2.97,205830.0,14088.0,23670.0,-0.04,7.35,11.50,19.69,99.0,0.60,1.46,14.64,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,60.0,H-MC,9.90,110828.0,10324.0,18785.0,0.13,10.27,16.95,28.96,79.0,0.55,0.79,47.73,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-41.30,47.0,M-SC,1.50,99172.0,7674.0,71077.0,0.11,8.39,71.67,86.07,223.0,0.11,0.71,44.02,XR,NTT,DURABLES
11,BANKINDIA,190.00,-26.71,70.0,H-MC,7.47,187836.0,8028.0,104380.0,0.58,4.47,55.57,62.52,88.0,0.08,1.34,40.07,XR,NTT,BANKS
86,WIPRO,420.00,-12.43,46.0,M-LC,5.35,154194.0,3249.0,106209.0,-0.37,2.15,68.88,72.51,53.0,0.03,1.10,8.30,XR,NTT,IT
37,INDIAMART,4850.92,-54.83,26.0,H-SC,10.71,125923.0,2587.0,131174.0,-0.63,2.10,104.17,108.45,119.0,0.02,0.90,24.92,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-18.84,35.0,H-LC,7.67,292140.0,-23702.0,136371.0,-0.13,-7.50,46.68,35.67,5.0,-0.17,2.08,5.31,X40N,ATH,FMCG
31,HINDUNILVR,2922.00,-10.50,37.0,H-LC,9.90,251150.0,-5913.0,41038.0,-0.42,-2.30,16.34,13.67,24.0,-0.14,1.79,17.64,XY25,NTT,FMCG
30,HCLTECH,1943.91,-1.94,39.0,H-LC,9.95,224055.0,-17869.0,81130.0,-0.19,-7.39,36.21,26.15,8.0,-0.22,1.59,5.88,X40,ATH,IT
5,ASIANPAINT,3465.66,-14.63,39.0,H-LC,7.65,215327.0,-36441.0,89641.0,-1.44,-14.47,41.63,21.13,27.0,-0.41,1.53,15.82,X40,ATH,PAINTS
42,ITC,452.00,-39.19,40.0,H-LC,2.06,196871.0,-3267.0,24156.0,-0.15,-1.63,12.27,10.44,4.0,-0.14,1.40,3.84,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-4.62,49.0,H-LC,1.56,236542.0,-9609.0,88822.0,-0.11,-3.90,37.55,32.18,16.0,-0.11,1.68,30.73,X200,ATH,IT
42,ITC,452.0,-39.19,40.0,H-LC,2.06,196871.0,-3267.0,24156.0,-0.15,-1.63,12.27,10.44,4.0,-0.14,1.40,3.84,X40,NTT,FMCG
65,SIEMENS,4671.5,0.60,54.0,H-LC,3.09,161810.0,-24285.0,71763.0,-0.13,-13.05,44.35,25.51,15.0,-0.34,1.15,19.24,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-13.66,47.0,H-LC,4.45,216200.0,5834.0,22939.0,-0.28,2.77,10.61,13.68,37.0,0.25,1.54,19.75,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.28,43.0,H-LC,4.56,272984.0,7558.0,49246.0,-0.15,2.85,18.04,21.40,11.0,0.15,1.94,10.85,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-28.04,42.0,H-LC,12.11,287161.0,-58795.0,133645.0,-0.24,-16.99,46.54,21.64,1.0,-0.44,2.04,1.73,X40,ATH,IT
42,ITC,452.00,-39.19,40.0,H-LC,2.06,196871.0,-3267.0,24156.0,-0.15,-1.63,12.27,10.44,4.0,-0.14,1.40,3.84,X40,NTT,FMCG
80,UNITDSPR,1644.00,-12.09,56.0,H-LC,7.30,230463.0,-3681.0,52315.0,-0.10,-1.57,22.70,20.77,86.0,-0.07,1.64,5.24,X40N,NTT,BREWERIES
83,VBL,671.64,-18.84,35.0,H-LC,7.67,292140.0,-23702.0,136371.0,-0.13,-7.50,46.68,35.67,5.0,-0.17,2.08,5.31,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.94,39.0,H-LC,9.95,224055.0,-17869.0,81130.0,-0.19,-7.39,36.21,26.15,8.0,-0.22,1.59,5.88,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.60,54.0,H-LC,3.09,161810.0,-24285.0,71763.0,-0.13,-13.05,44.35,25.51,15.0,-0.34,1.15,19.24,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,40.0,H-LC,2.06,196871.0,-3267.0,24156.0,-0.15,-1.63,12.27,10.44,4.0,-0.14,1.40,3.84,X40,NTT,FMCG
19,CIPLA,1795.00,-21.49,41.0,H-LC,5.13,209076.0,4576.0,36839.0,-0.07,2.24,17.62,20.25,10.0,0.12,1.49,11.17,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-14.63,39.0,H-LC,7.65,215327.0,-36441.0,89641.0,-1.44,-14.47,41.63,21.13,27.0,-0.41,1.53,15.82,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.66,47.0,H-LC,4.45,216200.0,5834.0,22939.0,-0.28,2.77,10.61,13.68,37.0,0.25,1.54,19.75,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-28.04,42.0,H-LC,12.11,287161.0,-58795.0,133645.0,-0.24,-16.99,46.54,21.64,1.0,-0.44,2.04,1.73,X40,ATH,IT
40,INFY,2275.00,-18.16,48.0,H-LC,8.11,318584.0,5696.0,165982.0,-0.12,1.82,52.10,54.87,3.0,0.03,2.27,8.59,X40,BTT,IT
42,ITC,452.00,-39.19,40.0,H-LC,2.06,196871.0,-3267.0,24156.0,-0.15,-1.63,12.27,10.44,4.0,-0.14,1.40,3.84,X40,NTT,FMCG
83,VBL,671.64,-18.84,35.0,H-LC,7.67,292140.0,-23702.0,136371.0,-0.13,-7.50,46.68,35.67,5.0,-0.17,2.08,5.31,X40N,ATH,FMCG
1,ABB,7934.00,-38.99,54.0,H-LC,8.61,255067.0,-6552.0,125774.0,0.29,-2.50,49.31,45.57,7.0,-0.05,1.82,8.53,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6894.44,57.0,L-SC,5.35,76717.0,-17093.0,93710.0,-0.79,-18.22,122.15,81.67,269.0,-0.18,0.55,48.57,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-41.65,54.0,L-SC,42.83,86400.0,-27149.0,67202.0,1.10,-23.91,77.78,35.27,268.0,-0.40,0.61,111.47,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.52,42.0,H-SC,11.75,87049.0,-13914.0,108446.0,-0.00,-13.78,124.58,93.63,148.0,-0.13,0.62,29.66,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.89,51.0,H-SC,12.68,94320.0,-3660.0,25259.0,-0.14,-3.74,26.78,22.05,152.0,-0.14,0.67,37.10,XR,ATH,FINANCE
50,NATIONALUM,244.55,-44.49,60.0,H-MC,9.90,110828.0,10324.0,18785.0,0.13,10.27,16.95,28.96,79.0,0.55,0.79,47.73,MH,ATH,METALS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1262.04,81.0,M-SC,10.07,185336.0,4193.0,41274.0,0.23,2.31,22.27,25.10,235.0,0.10,1.32,48.53,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.00,3829.73,59.0,M-SC,6.03,122449.0,-24919.0,76739.0,-0.34,-16.91,62.67,35.16,236.0,-0.32,0.87,25.07,XY24,NTT,AUTO
71,TANLA,1963.11,-16.22,69.0,H-SC,15.33,224344.0,-46417.0,376359.0,-0.45,-17.14,167.76,121.86,133.0,-0.12,1.60,75.50,AR,ATH,IT
81,VAIBHAVGBL,521.00,60.33,61.0,H-SC,5.96,139054.0,-43721.0,154795.0,-0.25,-23.92,111.32,60.77,125.0,-0.28,0.99,24.55,XR,NTT,JEWELLERY
60,SAIL,228.00,49.16,70.0,M-MC,9.75,235862.0,10900.0,156754.0,-0.24,4.85,66.46,74.53,192.0,0.07,1.68,38.66,XY24,BTT,STEEL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.19
1,25,43.60
2,50,74.75


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.07
LC    32.38
MC    23.56
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.42
X40      14.69
X40N     11.39
XR       11.31
XY25     10.00
AR        8.41
OX40N     7.81
X200      1.68
MH        1.65
X5K       1.43
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.38
H-LC    25.87
H-MC    20.59
M-SC    13.17
M-LC     5.47
M-MC     2.66
L-SC     1.52
L-LC     1.04
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.83,-5.60,40.45
IT,12.94,-15.57,78.59
FINANCE,10.04,-16.13,66.80
MISC,7.05,-16.27,80.90
BANKS,6.17,-15.05,75.57
PAINTS,5.62,-16.84,34.26
ELECTRICAL,5.49,-6.67,44.10
INSURANCE,3.71,-6.28,46.02
AC,3.55,2.85,13.42


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3107753.0,22
XR,1321731.0,14
AR,1152270.0,9
X40,782956.0,10
X40N,603063.0,9
OX40N,548988.0,10
XY25,434170.0,7
SR,249423.0,2
X5K,116683.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3663580.0,29
M-SC,1396964.0,17
H-MC,1213549.0,15
H-LC,1183687.0,15
M-LC,381386.0,4
M-MC,304409.0,2
L-SC,247035.0,3
L-MC,59550.0,1
L-LC,38505.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1226474.0      6
M-SC       XY24       794623.0      7
H-SC       AR         789436.0      5
           XR         774725.0      7
H-MC       XY24       559727.0      4
H-LC       X40        494554.0      5
M-MC       XY24       304409.0      2
H-SC       X40N       299475.0      4
           OX40N      260026.0      4
           SR         249423.0      2
H-LC       X40N       225525.0      3
H-MC       X40        216222.0      4
H-LC       AR         197537.0      2
H-MC       XY25       168119.0      2
M-SC       AR         165297.0      2
L-SC       XR         160912.0      2
M-LC       XY24       158494.0      2
M-SC       OX40N      134586.0      4
M-LC       X5K        116683.0      1
M-SC       XR         115955.0      2
           XY25       114323.0      1
H-LC       XY25       113223.0      3
M-LC       XR         106209.0      1
H-MC       XR         104380.0      1
H-LC       X200        88822.0      1
L-SC       OX40N       86123.0      1
H-MC       X40N        78063.0      2
M-SC       X40         72180.0      1
H-MC       OX40N       68253.0      1
H-LC       XY24        64026.0      1
H-SC       MH          64021.0      1
L-MC       XR          59550.0      1
L-LC       XY25        38505.0      1
H-MC       MH          18785.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
